# Introduction to Geant4.jl

Geant4.jl provides the Julia bindings for the [Geant4](https://geant4.web.cern.ch) particle transportation toolkit. It is using [CxxWrap.jl](https://github.com/JuliaInterop/CxxWrap.jl) package to wrap C++ types and functions to Julia. Since the Geant4 toolkit is rather large and complex, writing the wrapper code by hand is not really an option. For this we use the package [WrapIt](https://github.com/grasph/wrapit) that automates the generation of the wrapper code making use of the clang library.

Documentation of the concepts and how to write applications with the Geant4 toolkit can be found with the [Application Developer Guide](https://geant4-userdoc.web.cern.ch/UsersGuides/ForApplicationDeveloper/html/index.html) or the [Classes and Members reference guide](https://geant4.kek.jp/Reference/11.2.0/index.html) for a detailed description of each C++ class. In this tutorial we will only highlight the differences between the Julia and the C++ API. Therefore it is assumed some prior knowledge of the Geant4 C++ interfaces.

## Installation
The Geant4.jl package does no require any special installation. Stable releases are registered to the Julia general registry, and therefore, it can be deployed with the standard `Pkg` Julia package manager.

**Please note that the first time the `Geant4` package is added it may take some time since it downloads all the binary libraries and data files of Geant4** 

```julia
using Pkg
Pkg.add("Geant4")
```

## Interacting with the wrapped classes

Import the `Geant4` module. All the wrapped Geant4 classes are implicitly exported since they are prefixed by `G4`, and therefore the chances of a name clash with other Julia symbols is minimal.

### Object instantiation

We start by instantiating some very simple object of type `G4Box`.

Note that:
- The C++ constructor called is `G4Box::G4Box(const G4String &pName, G4double pX, G4double pY, G4double pZ)`
- The conversion of Julia type `Int64` to C++ `G4double` is implicit, as well as the `String` to `G4String` 

In [ ]:
using Geant4
box = G4Box("MyBox", 1, 2, 3)  # the C++ contructor called G4Box (const G4String &pName, G4double pX, G4double pY, G4double pZ)

The returned object is a pointer (`Ptr`) to the wrapped C++ object. The `CxxWrap` package encodes (with the type `Geant4.G4BoxAllocated`) that the object has been allocated from Julia and will by default be garbage collected when not needed.  

In [ ]:
using Test
@test typeof(box) == Geant4.G4BoxAllocated  # the type is indeed a G4BoxAllocated
@test box isa G4Box                         # but is also a G4Box (G4BoxAllocated inherits from G4Box)

subtypes(G4Box)                             # this shows all the subtypes of G4Box

To take the C++ pointer or the reference of a wrapped object, the `CxxWrap` module provides the functions `CxxPtr` and `CxxRef`. This is often needed to fullfil with the Geant4 C++ interfaces.    

In [ ]:
r_box =  CxxRef(box)
p_box =  CxxPtr(box)
@test r_box isa CxxRef{G4Box}  # reference
@test p_box isa CxxPtr{G4Box}  # pointer

The user can derefence a `CxxRef` or `CxxPtr` with the operator `[]`

In [ ]:
@show typeof(r_box[])
@show typeof(p_box[])
@test r_box[] == box
@test p_box[] == box

### Calling object methods
In julia, `methods` are instances of functions of a given name, of which the multi-dispatch functionality will select the best one that matches the actual argument types. To call a C++ method of a class we need to pass the wrapped object as first argument.

In [ ]:
vol = GetCubicVolume(box)   # In C++ this would be box.GetCubicVolume()
@test vol == 8 * GetXHalfLength(box) * GetYHalfLength(box) * GetZHalfLength(box) 

We can see the methods defined for the function `GetCubicVolume` using the Julia builtin function `methods()`

In [ ]:
methods(GetCubicVolume)

### Calling static class methods
In this case we do not have an object instance. The way it is done is by concatenating the class name with the method names with the symbol `!`. This is an example: 

In [ ]:
G4Random!getTheSeed()   # This will call the C++ class method G4Random::getTheSeed()

### Working in the inheritance
In Geant4 all solid types inherit from a common base class `G4VSolid`. The method `Clone` returns a pointer to this base class.  

In [ ]:
solid = Clone(box)
@test typeof(solid) == CxxPtr{G4VSolid}
@test CxxPtr(box) == box  # same instance
@test solid != box        # these are two diffrent instances
@test GetCubicVolume(solid) == GetCubicVolume(box)

### Object ownership
By default all objects instantiated by Julia will garbage collected (deleted) by Julia. this may pose problems with Geant4 since in many occasions the ownership is transferred implicitly to the toolkit, which will take care of doing a cleanup at the adequate moment. If we do nor pay attention we may get crashes (use a deleted objects and double deletions).

There are nevertheless some exceptions that have been added to simplify the interactions for the following classes: `G4PVPlacement`,
`G4LogicalVolume`, `G4PVReplica`, `G4Material`, `G4Isotope`, `G4Element`, `G4JLParticleGun`, 
                            `G4JLDetectorConstruction`, `G4JLGeneratorAction`, `G4JLRunAction`, `G4JLSensDet`, `G4JLWorkerInitialization`, `G4JLStateDependent`,
                            `G4LogicalSkinSurface`, `G4LogicalBorderSurface`, `G4OpticalSurface`

The following code show the behavior when things are not done correctly

In [ ]:
box1 = G4Box("box1", 1,1,3)                                   # construct boxes
box2 = G4Box("box2", 1,3,1)                                  
union = G4UnionSolid("union", CxxPtr(box1), CxxPtr(box2))     # construct a union solid with the same box
@show GetCubicVolume(union)
@show DistanceToIn(union, G4ThreeVector(10,10,10))


now force a GC

In [13]:
box1 = box2 = nothing
GC.gc()
@show GetCubicVolume(union)                     # G4BooleanSolid cashes the volume
# DistanceToIn(union, G4ThreeVector(10,10,10))  # this will probably crash the program

To avoid this problem the **user must transfer object ownership to the C++ side** using the provided function `move!()`. For example:

In [ ]:
box1 = G4Box("box1", 1,1,3)                                  # construct boxes
box2 = G4Box("box2", 1,3,1)                                  
union = G4UnionSolid("union", move!(box1), move!(box2))      # construct a union solid with the same box
@show GetCubicVolume(union)
@show DistanceToIn(union, G4ThreeVector(10,10,10));

forcing now a GC will still work

In [ ]:
box1 = box2 = zeros(1000)
GC.gc()
@show GetCubicVolume(union)                          # G4BooleanSolid cashes the volume
@show DistanceToIn(union, G4ThreeVector(10,10,10));  # this will probabluy crash the program